## ETL

In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import re

# Configuration
input_csv = '../data/raw.csv'
output_csv = '../data/transformed.csv'
chunksize = 100000


### 1. Define required columns

In [11]:
required_columns = [
    'title',
    'description',
    'max_salary',
    'med_salary',
    'min_salary',
    'pay_period',
    'formatted_work_type',
    'location',
    'applies',
    'original_listed_time',
    'remote_allowed',
    'views',
    'formatted_experience_level'
]

### 2. Using word boundaries to extract IT jobs

In [12]:
it_patterns = [
    r"\bIT\b",
    r"\bInformation Technology\b",
    r"\bSoftware\b",
    r"\bDeveloper\b",
    r"\bEngineer\b",
    r"\bProgrammer\b",
    r"\bTech\b",
    r"\bData\b",
    r"\bCloud\b",
    r"\bSystem\b",
    r"\bNetwork\b"
]

### 3. Exclude non-IT engineering roles

In [13]:
exclude_patterns = [
    r"\bBuilding\b",
    r"\bHVAC\b",
    r"\bMaintenance\b",
    r"\bFacilities?\b",
    r"\bStaff\b",
    r"\bConstruction\b",
    r"\bMechanical\b",
    r"\bElectrical\b",
    r"\bCivil\b",
    r"\bManufacturing\b",
    r"\bProject Manager\b",
    r"\bProject Engineer\b",
    r"\bQuality\b",
    r"\bSafety\b",
    r"\bLogistics\b",
    r"\bSupply Chain\b",
    r"\bField\b",
    r"\bService\b",
    r"\bSupport\b", 
    r"\bStructural\b",
    r"\bProcess\b", 
    r"\bDesign\b",
    r"\bInfrastructure\b",
    r"\bEnergy\b",
]

### 4. Exclude unwanted roles and check for IT job patterns

In [14]:
def is_it_job(title):
    if pd.isna(title):
        return False
    if any(re.search(pat, title, re.IGNORECASE) for pat in exclude_patterns):
        return False
    return any(re.search(pat, title, re.IGNORECASE) for pat in it_patterns)

### 5. Filter and select only required columns

In [15]:
with pd.read_csv(input_csv, chunksize=chunksize) as reader:
    for i, chunk in enumerate(reader):
        it_jobs = chunk[chunk['title'].apply(is_it_job)]

        for col in required_columns:
            if col not in it_jobs.columns:
                it_jobs[col] = pd.NA
        it_jobs = it_jobs[required_columns]

        if i == 0:
            it_jobs.to_csv(output_csv, index=False, mode='w')
        else:
            it_jobs.to_csv(output_csv, index=False, mode='a', header=False)

        print(f"Processed chunk {i+1}")

print("ETL complete. Output saved to", output_csv)

Processed chunk 1
Processed chunk 2
ETL complete. Output saved to ../data/transformed.csv


### 6.  Convert UNIX Timestamp to 24-Hour Time


In [16]:
with pd.read_csv(input_csv, chunksize=chunksize) as reader:
    for i, chunk in enumerate(reader):
        it_jobs = chunk[chunk['title'].apply(is_it_job)]

        for col in required_columns:
            if col not in it_jobs.columns:
                it_jobs[col] = pd.NA

        it_jobs = it_jobs[required_columns]

        # Convert original_listed_time from ms to datetime
        it_jobs['original_listed_time'] = pd.to_datetime(
            it_jobs['original_listed_time'], unit='ms', errors='coerce'
        )

        # Create new columns
        it_jobs['listed_date'] = it_jobs['original_listed_time'].dt.date
        it_jobs['listed_time_24hr'] = it_jobs['original_listed_time'].dt.time

        # Drop original_listed_time
        it_jobs = it_jobs.drop(columns=['original_listed_time'])

        if i == 0:
            it_jobs.to_csv(output_csv, index=False, mode='w')
        else:
            it_jobs.to_csv(output_csv, index=False, mode='a', header=False)

        print(f"Processed chunk {i+1}")

print("ETL complete. Output saved to", output_csv)


Processed chunk 1
Processed chunk 2
ETL complete. Output saved to ../data/transformed.csv


### 7. Standardization (Z-score normalization)

In [17]:


# Standardize numerical columns
scaler = StandardScaler()
numerical_cols = ['min_salary', 'max_salary']  # example numeric columns
it_jobs[numerical_cols] = scaler.fit_transform(it_jobs[numerical_cols])


### 8. Fill Missing Data

In [18]:
# Replace empty strings and non-standard nulls with actual NaN
it_jobs = it_jobs.replace(r'^\s*$', pd.NA, regex=True)
it_jobs = it_jobs.replace({'nan': pd.NA, 'NaN': pd.NA, 'none': pd.NA, 'None': pd.NA})

# Fill missing 'location' with a constant
if 'location' in it_jobs.columns:
    it_jobs['location'] = it_jobs['location'].fillna('Unknown')

# Convert 'min_salary' to numeric and fill with mean if possible
if 'min_salary' in it_jobs.columns:
    it_jobs['min_salary'] = pd.to_numeric(it_jobs['min_salary'], errors='coerce')
    if it_jobs['min_salary'].notna().any():
        mean_salary = it_jobs['min_salary'].mean()
        it_jobs['min_salary'] = it_jobs['min_salary'].fillna(mean_salary)
    else:
        it_jobs['min_salary'] = it_jobs['min_salary'].fillna(0)

# Fill 'pay_period' with mode or default
if 'pay_period' in it_jobs.columns:
    mode_pay_period = it_jobs['pay_period'].mode(dropna=True)
    if not mode_pay_period.empty:
        it_jobs['pay_period'] = it_jobs['pay_period'].fillna(mode_pay_period[0])
    else:
        it_jobs['pay_period'] = it_jobs['pay_period'].fillna('Not Specified')

# Fill 'remote_allowed' with False
if 'remote_allowed' in it_jobs.columns:
    it_jobs['remote_allowed'] = it_jobs['remote_allowed'].fillna(False)

# Forward and backward fill any remaining missing values
it_jobs = it_jobs.ffill().bfill()

# Check cleanup
print("Remaining missing values:\n", it_jobs.isna().sum())
print("ETL complete. Output saved to", output_csv)

Remaining missing values:
 title                         0
description                   0
max_salary                    0
med_salary                    0
min_salary                    0
pay_period                    0
formatted_work_type           0
location                      0
applies                       0
remote_allowed                0
views                         0
formatted_experience_level    0
listed_date                   0
listed_time_24hr              0
dtype: int64
ETL complete. Output saved to ../data/transformed.csv
